In [1]:
import pandas as pd
import openpyxl
import numpy as np

In [2]:
satzart_dict = {
    "10": "Land",
    "20": "Regierungsbezirk",
    "30": "Region",
    "40": "Kreis",
    "50": "Gemeindeverband",
    "60": "Gemeinde",
}

In [3]:
textkennzeichen_dict = {
    "41": "Kreisfreie Stadt",
    "42": "Stadtkreis",
    "43": "Kreis",
    "44": "Landkreis",
    "45": "Regionalverband",
    "50": "Verbandsfreie Gemeinde",
    "51": "Amt",
    "52": "Samtgemeinde",
    "53": "Verbandsgemeinde",
    "54": "Verwaltungsgemeinschaft",
    "55": "Kirchspielslandgemeinde",
    "56": "Verwaltungsverband",
    "58": "Erfüllende Gemeinde",
    "60": "Markt",
    "61": "Kreisfreie Stadt",
    "62": "Stadtkreis",
    "63": "Stadt",
    "64": "Kreisangehörige Gemeinde",
    "65": "gemeindefreies Gebiet-bewohnt",
    "66": "gemeindefreies Gebiet-unbewohnt",
    "67": "Große Kreisstadt",
}

In [4]:
col_names = [
    "satzart", "textkennzeichen", "ars_land", "ars_rb",
    "ars_kreis", "ars_vb", "ars_gemeinde", "gemeindename",
    "flaeche_km2", "bev_geseamt", "bev_maennl", "bev_weibl", 
    "bev_pro_km2", "plz", "longitude", "latitude", "reisegebiet_id",
    "reisegebiet_name", "urbanisierung_id", "urbanisierung_cat",
]

dtype_dict = dict(zip(col_names, [object for i in range(len(col_names))]))

In [6]:
# read the content
filepath = "../data/raw/GV100AD/AuszugGV4QAktuell.xlsx"
data = pd.read_excel(
    filepath, 
    sheet_name="Onlineprodukt_Gemeinden",
    skiprows=6,
    header=None,
    engine="openpyxl",
    usecols="A:T",
    names=col_names,
    dtype=dtype_dict,
)
# filter for the rows that contain a satzart value
data = data[data["satzart"].isin(list(satzart_dict.keys()))]

In [7]:
# prepare ars codes
data["_ars_rb"] = data['ars_land'] + data['ars_rb']
data["_ars_kreis"] = data['ars_land'] + data['ars_rb'] + data["ars_kreis"]
data["_ars_vb"] = data['ars_land'] + data['ars_rb'] + data["ars_kreis"] + data["ars_vb"]
data["_ars_gemeinde"] = data['ars_land'] + data['ars_rb'] + data["ars_kreis"] + data["ars_vb"] + data["ars_gemeinde"]
# convert to dict
land_dict = data[data.satzart == "10"].set_index("ars_land")["gemeindename"].to_dict()
rb_dict = data[data.satzart == "20"].set_index("_ars_rb")["gemeindename"].to_dict()
kreis_dict = data[data.satzart == "40"].set_index("_ars_kreis")["gemeindename"].to_dict()
vb_dict = data[data.satzart == "50"].set_index("_ars_vb")["gemeindename"].to_dict()
# map dicts on ars codes
data['land'] = data.ars_land.map(land_dict)
data['rb'] = data._ars_rb.map(rb_dict)
data['kreis'] = data._ars_kreis.map(kreis_dict)
data['vb'] = data._ars_vb.map(vb_dict)
data['kennzeichen'] = data.textkennzeichen.map(textkennzeichen_dict)

In [8]:
# filter for gemeinde level data and collection of cols
col_selection = [
    'land', 'rb', 'kreis', 'vb', 'gemeindename', '_ars_gemeinde' ,'kennzeichen',
    'flaeche_km2', 'bev_geseamt', 'bev_maennl', 'bev_weibl', 'bev_pro_km2', 
    'plz', 'longitude', 'latitude', 'reisegebiet_name', 'urbanisierung_cat',
]
data = data.dropna(subset=["ars_gemeinde"])[col_selection]

In [9]:
# export dataframe to csv
filepath = '../data/processed/gv100ad.csv'
data.to_csv(filepath, index=False)